# Reviewing previous session

In [ ]:
import tensorflow as tf

In [2]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    

In [7]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
X_data = housing.data
Y_data = np.reshape(housing.target, [-1, 1])
print("Shape of X", X_data.shape)
print("Shape of Y", Y_data.shape)

##################################################################

tf.reset_default_graph()
tf.set_random_seed(42)
learning_rate = 0.0000005
epochs = 5000
n_data = len(X_data)
X = tf.constant(X_data, dtype=tf.float32, name='X')
Y = tf.constant(Y_data, dtype=tf.float32, name='Y')
W = tf.Variable(tf.random.normal(shape=[8, 1], mean=0.0, stddev=0.01, name="W"))
Y_hat = tf.matmul(X, W, name="Y_hat")
errors = Y_hat - Y
squared_errors = 0.5 * tf.square(errors, name='squared_errors')
MSE = tf.reduce_mean(squared_errors, name='MSE')
optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="optimizer")
update_W = optimizer.minimize(MSE)
init_operation = tf.global_variables_initializer()
show_graph(tf.get_default_graph())
# with tf.Session() as ses:
#     ses.run(init_operation)
#     for i in range(epochs):
#         _, mse = ses.run([update_W, MSE])
#         if i % 1000 == 0:
#             print("Epoch= ", i, "Mse = ", mse, end="\n")
    
#     print()
        
#     print(ses.run(W))


Shape of X (20640, 8)
Shape of Y (20640, 1)


In [ ]:
show_graph(tf.get_default_graph())

# Tensorflow Model Template

In [ ]:
class DataLoader():
    def __init__(self):
        pass
    
    def get_train_batch():
        pass
    
    def get_val_batch():
        pass
    
    
class Model():
    def __init__(self):
        pass
    
    def create_graph(self):
        pass
    
    def train(self):
        pass
    
    def save(self):
        pass
    
    def load(self):
        pass
            

## Writing the DataLoader Class

In [6]:
class DataLoader():
    def __init__(self, batch_size=32):
        self.b = batch_size
        housing_data = fetch_california_housing()
        self.X = housing.data
        self.Y = np.reshape(housing.target, [-1, 1])
        self.Y = np.log(self.Y)
        self.X = self.X / np.std(self.X, axis=0)
        self.n = len(self.X)
        self.X_train, self.Y_train = self.X[: self.n // 2], self.Y[: self.n // 2]
        self.X_val, self.Y_val = self.X[self.n // 2:], self.Y[self.n // 2:]
        self.n_train, self.n_val = len(self.X_train), len(self.X_val)
        self.tp, self.vp = 0, 0
        
    
    def get_train_batch(self):
        if self.tp >= self.n_train:
            self.tp = 0
        xb, yb = self.X_train[self.tp: self.tp + self.b], self.Y_train[self.tp: self.tp + self.b]
        self.tp += self.b
        return xb, yb
    
    def get_val_batch(self):
        if self.vp >= self.n_train:
            self.vp = 0
        xb, yb = self.X_val[self.vp: self.vp + self.b], self.Y_val[self.vp: self.vp + self.b]
        self.vp += self.b
        return xb, yb
    
dataloader = DataLoader(batch_size=5)

x, y = dataloader.get_train_batch()
print(x.shape, y.shape)    

(5, 8) (5, 1)


## Writing the Model Class

### Initializing

In [ ]:
import os
import datetime

class Model():
    def __init__(self, name='Model', dataloader=None):
        self.name = name
        self.dataloader = dataloader
        
        self.save_path = os.path.join("SavedModels/", self.name)
        try:
            os.mkdir(self.save_path)
        except:
            pass
        
        init_time = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
        self.log_path = os.path.join("log/" + self.name + "-run-" + init_time)
        
        self.create_graph()
        self.ses = tf.Session()
        
    
    def create_graph(self):
        pass
    
    def train(self):
        pass
    
    def save(self):
        pass
    
    def load(self):
        pass
            
        
M1 = Model("M1")

### Creating the Graph

#### Namespaces in Tensorflow

In [18]:
tf.reset_default_graph()
x = tf.Variable(2, name='x')
y = tf.Variable(3, name='y')
z = x * y + x * x + 3
show_graph(tf.get_default_graph())

tf.reset_default_graph()
with tf.variable_scope("scope1"):
    x = tf.Variable(2, name='x')
    y = tf.Variable(3, name='y', trainable=False)
    z = x * y + x * x + 3
    
# print(tf.global_variables())
    
with tf.variable_scope("scope2"):
    x = tf.Variable(4, name='x')
    y = tf.Variable(5, name='y', trainable=False)
    z = x * y + x * x + 3
    
# 
with tf.variable_scope("scope3"):
    x = tf.Variable(6, name='x')
    y = tf.Variable(7, name='y', trainable=False)
    z = x * y + x * x + 3

# show_graph(tf.get_default_graph())

# x
# tf.global_variables()
# tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='scope1')
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='scope1')


[<tf.Variable 'scope1/x:0' shape=() dtype=int32_ref>]

In [21]:
tf.reset_default_graph()
with tf.variable_scope("A"):
    z = tf.Variable(2, name="x")
    with tf.variable_scope("B"):
        x = tf.Variable(2, name="x")
        
x, z

(<tf.Variable 'A/B/x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'A/x:0' shape=() dtype=int32_ref>)

#### Reusing the same function

In [ ]:
tf.reset_default_graph()
X1 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X1')
h1_X1 = tf.layers.dense(X1, 32, name='layer1_X1')
h2_X1 = tf.layers.dense(h1_X1, 8, name='layer2_X1')

X2 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X2')
h1_X2 = tf.layers.dense(X2, 32, name='layer1_X2')
h2_X2 = tf.layers.dense(h1_X2, 8, name='layer2_X2')

In [ ]:
tf.reset_default_graph()
X1 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X1')
X2 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X2')

def transform(x):
    h1 = tf.layers.dense(x, 32, name='layer1')
    h2 = tf.layers.dense(h1, 8, name='layer2')
    return h2

X1_transformed = transform(X1)
X2_transformed = transform(X2)
show_graph(tf.get_default_graph())

In [27]:
tf.reset_default_graph()
X1 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X1')
X2 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X2')
X3 = tf.placeholder(dtype=tf.float32, shape=[None, 128], name='X3')


def transform(x):
    with tf.variable_scope("transform", reuse=tf.AUTO_REUSE):
        h1 = tf.layers.dense(x, 32, name='layer1')
        h2 = tf.layers.dense(h1, 8, name='layer2')
    return h2

X1_transformed = transform(X1)
X2_transformed = transform(X2)
X3_transformed = transform(X3)

# tf.trainable_variables()
show_graph(tf.get_default_graph())

In [ ]:
import os
import datetime

class Model():
    def __init__(self, name='Model', dataloader=None):
        self.name = name
        self.dataloader = dataloader
        
        self.save_path = os.path.join("SavedModels/", self.name)
        try:
            os.mkdir(self.save_path)
        except:
            pass
        
        init_time = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
        self.log_path = os.path.join("log/" + self.name + "-run-" + init_time)
        
        self.create_graph()
        self.ses = tf.Session()
        
    
    def create_graph(self):
        tf.reset_default_graph()
        tf.set_random_seed(42)
        with tf.variable_scope(self.name):
            with tf.variable_scope("Placeholders"):
                self.x_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 8]) # dataloader must provide the shape
                self.y_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 1])
            
            with tf.variable_scope("Inference"):
                h1 = tf.layers.dense(inputs=self.x_placeholder, units=16, activation=tf.nn.sigmoid, name='layer1')
                h2 = tf.layers.dense(inputs=h1, units=8, activation=tf.nn.sigmoid, name='layer2')
                self.y_hat = tf.layers.dense(inputs=h2, units=1, name='y_hat')
                
            with tf.variable_scope("Optimization"):
                self.loss = tf.losses.mean_squared_error(labels=self.y_placeholder, predictions=self.y_hat)
                self.global_step = tf.train.get_or_create_global_step()
                learning_rate = tf.train.exponential_decay(learning_rate=1e-4,
                                                           decay_steps=10000,
                                                           decay_rate=0.5,
                                                           global_step=self.global_step,
                                                           name='learning_rate')
                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                self.train_operation = optimizer.minimize(self.loss, global_step=self.global_step)
                
                
    def train(self):
        pass
    
    def save(self):
        pass
    
    def load(self):
        pass
            

dataloader = DataLoader(batch_size=16)
M1 = Model(name="M1", dataloader=dataloader)
# show_graph(tf.get_default_graph())

### Train, Save, Load

In [28]:
import os
import datetime

class Model():
    def __init__(self, name='Model', dataloader=None):
        self.name = name
        self.dataloader = dataloader
        
        self.save_path = os.path.join("SavedModels/", self.name)
        try:
            os.mkdir(self.save_path)
        except:
            pass
        
        init_time = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
        self.log_path = os.path.join("log/" + self.name + "-run-" + init_time)
        
        self.create_graph()
        self.ses = tf.Session()
        
    
    def create_graph(self):
        tf.reset_default_graph()
        tf.set_random_seed(42)
        with tf.variable_scope(self.name):
            with tf.variable_scope("Placeholders"):
                self.x_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 8]) # dataloader must provide the shape
                self.y_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 1])
            
            with tf.variable_scope("Inference"):
                h1 = tf.layers.dense(inputs=self.x_placeholder, units=16, activation=tf.nn.sigmoid, name='layer1')
                h2 = tf.layers.dense(inputs=h1, units=8, activation=tf.nn.sigmoid, name='layer2')
                self.y_hat = tf.layers.dense(inputs=h2, units=1, name='y_hat')
                
            with tf.variable_scope("Optimization"):
                self.loss = tf.losses.mean_squared_error(labels=self.y_placeholder, predictions=self.y_hat)
                self.global_step = tf.train.get_or_create_global_step()
                learning_rate = tf.train.exponential_decay(learning_rate=1e-4, # 1e-3
                                                           decay_steps=1000,
                                                           decay_rate=0.5,
                                                           global_step=self.global_step,
                                                           name='learning_rate')
                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                self.train_operation = optimizer.minimize(self.loss, global_step=self.global_step)
                
            with tf.variable_scope("Init_save"):
                self.variable_initializer = tf.global_variables_initializer()
                self.saver = tf.train.Saver()
                
                
    def train(self, init=True, steps=10000):
        if init:
            self.ses.run(self.variable_initializer)
        step = 0
        while step < steps:
            x_train, y_train = self.dataloader.get_train_batch()
            x_val, y_val = self.dataloader.get_val_batch()
            val_loss = self.ses.run(self.loss, feed_dict={self.x_placeholder: x_val, self.y_placeholder: y_val})
            train_loss, step, _ = self.ses.run([self.loss, self.global_step, self.train_operation],
                                               feed_dict={self.x_placeholder: x_train, self.y_placeholder: y_train})
            if step % 1000 == 0:
                print("Step = ", step)
                print("\tTrain loss = ", train_loss)
                print("\tValidation loss = ", val_loss)
    
    def save(self):
        self.saver.save(self.ses, save_path=self.save_path + "/" + self.name + '.ckpt')
    
    def load(self):
        self.saver.restore(self.ses, save_path=self.save_path + "/" + self.name + '.ckpt')

            

dataloader = DataLoader(batch_size=16)
M1 = Model(name="M1", dataloader=dataloader)
# show_graph(tf.get_default_graph())
M1.train(init=True, steps=4000)
M1.save()
# M1.load()


Step =  0
	Train loss =  1.4474123
	Validation loss =  1.9790204
Step =  1000
	Train loss =  1.0408561
	Validation loss =  1.4456122
Step =  2000
	Train loss =  0.01940269
	Validation loss =  0.27312785
Step =  3000
	Train loss =  1.5633396
	Validation loss =  1.369827
Step =  4000
	Train loss =  0.77247214
	Validation loss =  0.24396871


### Adding Summaries

In [29]:
import os
import datetime

class Model():
    def __init__(self, name='Model', dataloader=None):
        self.name = name
        self.dataloader = dataloader
        
        self.save_path = os.path.join("SavedModels/", self.name)
        try:
            os.mkdir(self.save_path)
        except:
            pass
        
        init_time = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
        self.log_path = os.path.join("log/" + self.name + "-run-" + init_time)
        
        self.create_graph()
        self.ses = tf.Session()
        
    
    def create_graph(self):
        tf.reset_default_graph()
        tf.set_random_seed(42)
        with tf.variable_scope(self.name):
            with tf.variable_scope("Placeholders"):
                self.x_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 8]) # dataloader must provide the shape
                self.y_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 1])
            
            with tf.variable_scope("Inference"):
                h1 = tf.layers.dense(inputs=self.x_placeholder, units=16, activation=tf.nn.sigmoid, name='layer1')
                h2 = tf.layers.dense(inputs=h1, units=8, activation=tf.nn.sigmoid, name='layer2')
                self.y_hat = tf.layers.dense(inputs=h2, units=1, name='y_hat')
                
            with tf.variable_scope("Optimization"):
                self.loss = tf.losses.mean_squared_error(labels=self.y_placeholder, predictions=self.y_hat)
                self.global_step = tf.train.get_or_create_global_step()
                learning_rate = tf.train.exponential_decay(learning_rate=1e-4, # 1e-3
                                                           decay_steps=1000,
                                                           decay_rate=0.5,
                                                           global_step=self.global_step,
                                                           name='learning_rate')
                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                self.train_operation = optimizer.minimize(self.loss, global_step=self.global_step)
                
            with tf.variable_scope("Init_save"):
                self.variable_initializer = tf.global_variables_initializer()
                self.saver = tf.train.Saver()
                
            with tf.variable_scope("Summary"):
                self.train_summary_writer = tf.summary.FileWriter(self.log_path + "train", tf.get_default_graph())
                self.validation_summary_writer = tf.summary.FileWriter(self.log_path + "validation")
                self.loss_summary = tf.summary.scalar(name="Mean_Squared_Loss", tensor=self.loss)
                
                
    def train(self, init=True, steps=10000):
        if init:
            self.ses.run(self.variable_initializer)
        step = 0
        while step < steps:
            x_train, y_train = self.dataloader.get_train_batch()
            x_val, y_val = self.dataloader.get_val_batch()
            val_loss_summary = self.ses.run(self.loss_summary, feed_dict={self.x_placeholder: x_val, self.y_placeholder: y_val})
            train_loss_summary, step, _ = self.ses.run([self.loss_summary, self.global_step, self.train_operation],
                                               feed_dict={self.x_placeholder: x_train, self.y_placeholder: y_train})
            self.train_summary_writer.add_summary(train_loss_summary, step)
            self.validation_summary_writer.add_summary(val_loss_summary, step)
            if step % 1000 == 0:
                print("Step = ", step)
                
        self.train_summary_writer.close()
        self.validation_summary_writer.close()
                
    
    def save(self):
        self.saver.save(self.ses, save_path=self.save_path + "/" + self.name + '.ckpt')
    
    def load(self):
        self.saver.restore(self.ses, save_path=self.save_path + "/" + self.name + '.ckpt')

            

## Running Final Model

In [30]:
dataloader = DataLoader(batch_size=64)
M1 = Model(name="M1", dataloader=dataloader)
# show_graph(tf.get_default_graph())
M1.train(init=True, steps=4000)
M1.save()
# M1.load()

Step =  0
Step =  1000
Step =  2000
Step =  3000
Step =  4000
